# Tarefa 4

Alunos:

* Andreza - RA: 164213
* Gil - RA: 225323
* Yan - RA: 118982

In [121]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [132]:
df = pd.read_csv("dados4.csv")
X = df.drop(['V15'], axis=1)
y = df.filter(['V15'], axis=1)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
0,1,22.08,11.46,2,k,bb,1.585,0,0,0,1,g,100,1213,0
1,0,22.67,7.00,2,c,bb,0.165,0,0,0,0,g,160,1,0
2,0,29.58,1.75,1,k,bb,1.250,0,0,0,1,g,280,1,0
3,0,21.67,11.50,1,j,j,0.000,1,1,11,1,g,0,1,1
4,1,20.17,8.17,2,aa,bb,1.960,1,1,14,0,g,60,159,1


## Preprocessamento

In [133]:
X = pd.get_dummies(df, columns=['V5', 'V6', 'V12'])

,V1,V2,V3,V4,V7,V8,V9,V10,V11,V13,...,V6_dd,V6_ff,V6_h,V6_j,V6_o,V6_v,V6_z,V12_g,V12_p,V12_s
0,1,22.08,11.46,2,1.585,0,0,0,1,100,...,0,0,0,0,0,0,0,1,0,0


In [134]:
X = StandardScaler().fit_transform(X)
X = pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0.688737,-0.801052,1.347111,0.54295,-0.190906,-1.047504,-0.864196,-0.493887,1.087908,-0.488358,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.32249,-0.108306,-0.300079


## Métodos

In [175]:
methods = {
    'Logistic_Regression': LogisticRegression(penalty='none'),
    'Logistic_Regression_l2': LogisticRegression(penalty='l2'),
    'LDA': LinearDiscriminantAnalysis(),
    'QDA': QuadraticDiscriminantAnalysis(),
    'SVM_Linear': LinearSVC(),
    'SVM_RBF': SVC(kernel='rbf'),
    'Naive_Bayes': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'MLP': MLPClassifier(),
    'Decision_Tree': DecisionTreeClassifier(),
    'Random_Forest': RandomForestClassifier(),
    'GBM': GradientBoostingClassifier(),
}

params={
    'Logistic_Regression': {},
    'Logistic_Regression_l2': {'C': 10**(np.random.uniform(-3, 3, 10))},
    'LDA': {},
    'QDA': {},
    'SVM_Linear': {'C': 2**(np.random.uniform(-5, 15, 10))},
    'SVM_RBF': {'C': 2**(np.random.uniform(-5, 15, 10)),
                'gamma': 2**(np.random.uniform(-9, 3, 10))},
    'Naive_Bayes': {},
    'KNN': {'n_neighbors': np.random.randint(1, 301, 10)},
    'MLP': {'hidden_layer_sizes': np.arange(5, 21, 3)},
    'Decision_Tree': {'ccp_alpha': np.random.uniform(0.04, 0.0, 10)},
    'Random_Forest': {'n_estimators': [10, 100, 1000],
                      'max_features': [5, 8, 10]},
    'GBM': {'n_estimators':np.random.randint(5, 100, 10),
            'learning_rate': np.random.uniform(0.01, 0.3, 10),
            'max_depth': [2, 3]}
}

## Iterações

In [176]:
results = {
    'clf': [],
    'mean_auc_outer': []#,
#    'best_params': []
}

for method in methods:
    print(method)
    auc_outer = []
    for iteration in range(4):
        print(iteration)
        X_train, X_test, y_train, y_test = train_test_split(X, np.ravel(y), test_size=0.3)
        clf = GridSearchCV(
                    methods[method],
                    params[method],
                    cv=StratifiedKFold(n_splits=3),
                    scoring='roc_auc',
                    refit=True)
        clf.fit(X_train,y_train)
        # get prediction on test (outer loop)
        y_pred = clf.predict(X_test)
        # calculate auc and save
        auc = roc_auc_score(y_test, y_pred)
        auc_outer.append(auc)
#        break
    mean_auc = np.mean(auc_outer)
    results['clf'].append(method)
    results['mean_auc_outer'].append(mean_auc)
#    results['best_params'].append(clf.best_params_)
#    break


Logistic_Regression
0
1
2
3
Logistic_Regression_l2
0
1
2
3
LDA
0
1
2
3
QDA
0
1


/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/hom

2
3


/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/hom

SVM_Linear
0


...

2


/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/yan/anaconda3/lib/python3.8/site-packages/skle

3


/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/yan/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/yan/anaconda3/lib/python3.8/site-packages/skle

Decision_Tree
0
1
2
3
Random_Forest
0
1
2
3
GBM
0
1
2
3


In [177]:
results_df = pd.DataFrame(results)
results_df

,clf,mean_auc_outer
0,Logistic_Regression,1.000000
1,Logistic_Regression_l2,1.000000
2,LDA,1.000000
3,QDA,0.989449
4,SVM_Linear,1.000000
5,SVM_RBF,0.983007
6,Naive_Bayes,0.989921
7,KNN,0.927029
8,MLP,0.977153
9,Decision_Tree,1.000000
